# First Name: Soumyadeep 
# Last Name: Sarkar

# Import Libraries  

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import scipy

# Import Data

In [ ]:
pd.set_option('display.float_format', lambda x:'%.2f'%x)
bottle = pd.read_csv('bottle.csv', low_memory=False)
bottle.head()

# Data management 

In [ ]:
# Convert all variables to numeric.
bottle["T_degC"] = pd.to_numeric(bottle["T_degC"], errors="coerce")
bottle["O2Sat"] = pd.to_numeric(bottle["O2Sat"], errors="coerce")
bottle["Phaeop"] = pd.to_numeric(bottle["Phaeop"], errors="coerce")

In [ ]:
# Obtain subset of temperature up to 25 degrees.
# This limiting is also to cut off outliers.
sub1 = bottle[(bottle["T_degC"] > 0) & (bottle["T_degC"] < 25)].copy()
sns.distplot(sub1["T_degC"].dropna(), kde=False);

In [ ]:
# Obtain subset of oxygen saturation up to 140%.
# This value was chosen because it is the maximum 
# value for which a significant number of records exist, readings above this are considered outliers.
sub2 = sub1[(sub1["O2Sat"] < 140)].copy()
sns.distplot(sub2["O2Sat"].dropna(), kde=False);

In [ ]:
# Obtain subset of phaeophytin concentration between 0 and 1.
# Again, this cutting of values is to reduce outliers.
sub3 = sub2[(sub2["Phaeop"] > 0) & (sub2["Phaeop"] < 1)].copy()
sns.distplot(sub3["Phaeop"].dropna(), kde=False);

In [ ]:
# Obtain another subset containing only the required variables and with no null values.
sub4 = sub3[["T_degC", "O2Sat", "Phaeop"]].dropna()

# Correlation between each explantory variable and response variable (y=total_cases)

In [ ]:
# There is a strong positive correlation between temperature and oxygen saturation.
print("Association between temperature and oxygen saturation.")
print(scipy.stats.pearsonr(sub4["T_degC"], sub4["O2Sat"]))

In [ ]:
# There is a weak positive correlation between phaophytin concentration and oxygen saturation.
print("Association between temperature and oxygen saturation.")
print(scipy.stats.pearsonr(sub4["Phaeop"], sub4["O2Sat"]))

# Scatter plot between each explantory variable and response variable (y=total_cases)

In [ ]:
# Scatter plot of correlation between temperature (x) and oxygen saturation (y).
plt.figure()
scat = sns.regplot(x="T_degC", y="O2Sat", fit_reg=True, order=1, data=sub4)
plt.xlabel("Temperature")
plt.ylabel("Oxygen Saturation")
plt.title("Scatterplot for the Association Between Temperature\nand Oxygen Saturation");

In [ ]:
# Scatter plot of correlation between phaeophytin concentration (x) and oxygen saturation (y).
plt.figure()
scat = sns.regplot(x="Phaeop", y="O2Sat", fit_reg=True, data=sub4)
plt.xlabel("Phaeophytin Concentration")
plt.ylabel("Oxygen Saturation")
plt.title("Scatterplot for the Association Between Phaeophytin\nConcentration and Oxygen Saturation");

In [ ]:
# Center all variables by subtracting mean values.
sub4["T_degC_c"] = (sub4["T_degC"] - sub4["T_degC"].mean())
sub4["O2Sat_c"] = (sub4["O2Sat"] - sub4["O2Sat"].mean())
sub4["Phaeop_c"] = (sub4["Phaeop"] - sub4["Phaeop"].mean())

# Regression Analysis

In [ ]:
# Perform regression analysis of how O2Sat_c is affected by T_degC_c and Phaeop_c.
reg1 = smf.ols("O2Sat_c ~ T_degC_c + Phaeop_c", data=sub4).fit()
print (reg1.summary())

In [ ]:
# R-squared: 0.752
# p-value: 0.0
# equation: O2Sat_c = -8.618e-13 + 7.0153(T_degC_c) + 25.5296(Phaeop_c)

# qq plot 

In [ ]:
# qqplot of regression analysis.
# The points fit the line well for -2 < x < 2.
import statsmodels.api as sm
fig1 = sm.qqplot(reg1.resid, line="r")

# standardized residual plots

In [ ]:
# Residual plot of regression analysis. 
# The points appear to be evenly distributed above and below zero.
stdres = pd.DataFrame(reg1.resid_pearson)

plt.figure()
plt.plot(stdres, 'o', ls='None')
l = plt.axhline(y=0, color='r')
plt.ylabel('Standardized Residual')
plt.xlabel('Observation Number')
plt.title("Residual Plot of Regression Analysis\nfor O2Sat_c ~ T_degC_c + Phaeop_c");

In [ ]:
# Calculate percentage of residuals with more than 2 standard deviation.
# Value is < 5%, which is the maximum that can be considered a good fit.
percentage_over2sd = (np.count_nonzero( stdres[0] > 2) + np.count_nonzero( stdres[0] < -2))/len(stdres)*100
print (percentage_over2sd)

In [ ]:
# Calculate percentage of residuals with more than 2.5 standard deviation.
# Value is < 1%, which is the maximum that can be considered a good fit.
percentage_over2_5sd = (np.count_nonzero( stdres[0] > 2.5) + np.count_nonzero( stdres[0] < -2.5))/len(stdres)*100
print (percentage_over2_5sd)